In [1]:
import os
import joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder, QuantileTransformer
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline

from model_settings import ms
ms.find_root(Path())
models_dir = os.path.join(ms.root,ms.trained_models)
models = [f for f in os.listdir(models_dir) if f.find('Legacy')==-1]
models

['2024-11-26 222009133940 cboe spx relative asian',
 '2024-11-26 222433412525 cboe spx relative barrier']

In [2]:
model_dir = os.path.join(models_dir,models[0])
price_name = model_dir[model_dir.rfind(' ',0)+1:]+"_price"
pickle = [os.path.join(model_dir,f) for f in os.listdir(model_dir) if f.endswith('.pkl')][0]
model = joblib.load(pickle)
mlp = model['model']
mlp

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('StandardScaler',
                                                  StandardScaler(),
                                                  ['days_to_maturity',
                                                   'fixing_frequency',
                                                   'past_fixings',
                                                   'risk_free_rate',
                                                   'dividend_rate', 'kappa',
                                                   'theta', 'rho', 'eta', 'v0',
                                                   'relative_spot']),
                                                 ('OneHotEncoder',
                                                  OneHotEncoder(sparse_output=False),
                                                  ['averaging_type', 'w'])])),
                ('regressor',
                 MLPRegressor(alpha=0.01, hidden_layer_sizes=(13, 13),
                              learning_rate='adaptive', learning_rate_init=0.1,
                              max_iter=500, random_state=1312, solver='sgd',
                              warm_start=True))])

In [3]:
dates = model['development_dates']
days = dates.dt.round('D').drop_duplicates().reset_index(drop=True)
days

0    2012-01-03
1    2012-01-05
2    2012-01-06
3    2012-01-10
4    2012-01-18
        ...    
95   2012-10-15
96   2012-10-23
97   2012-10-31
98   2012-11-02
99   2012-11-05
Name: date, Length: 100, dtype: datetime64[ns]

In [4]:
latest = days.iloc[1:2]
ante = days.iloc[2:3]
print(f"test:\n{ante},\n\ntrain:\n{latest}")

test:
2   2012-01-06
Name: date, dtype: datetime64[ns],

train:
1   2012-01-05
Name: date, dtype: datetime64[ns]


In [5]:
print(mlp)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('StandardScaler',
                                                  StandardScaler(),
                                                  ['days_to_maturity',
                                                   'fixing_frequency',
                                                   'past_fixings',
                                                   'risk_free_rate',
                                                   'dividend_rate', 'kappa',
                                                   'theta', 'rho', 'eta', 'v0',
                                                   'relative_spot']),
                                                 ('OneHotEncoder',
                                                  OneHotEncoder(sparse_output=False),
                                                  ['averaging_type', 'w'])])),
                ('regressor',
                 MLPRegressor(alpha=0.01, hidden_layer_sizes=(13

In [ ]:
param_grid = {
    # 'regressor__activation': [
    #     'identity', 
    #     'logistic', 
    #     'tanh', 
    #     'relu'
    # ],
    # 'regressor__solver': ['lbfgs', 'sgd',],
    'regressor__alpha': [0.0001, 0.001, 0.01, 0.1],
    'regressor__batch_size': ['auto', 32, 64, 128],
    'regressor__learning_rate': ['constant', 'invscaling', 'adaptive'],
    'regressor__learning_rate_init': [0.001, 0.01, 0.1],
    # 'regressor__power_t': [0.5, 0.25],
    # 'regressor__max_iter': [200, 500, 1000],
    # 'regressor__shuffle': [True, False],
    # 'regressor__tol': [1e-4, 1e-3, 1e-2],
    # 'regressor__warm_start': [False, True],
    # 'regressor__momentum': [0.9, 0.8, 0.7],
    # 'regressor__nesterovs_momentum': [True, False],
    # 'regressor__early_stopping': [False, True],
    # 'regressor__validation_fraction': [0.1, 0.15, 0.2],
    # 'regressor__beta_1': [0.9, 0.85],
    # 'regressor__beta_2': [0.999, 0.99],
    # 'regressor__epsilon': [1e-8, 1e-6],
    # 'regressor__n_iter_no_change': [10, 20],
    # 'regressor__max_fun': [15000, 20000]
}

grid_search = GridSearchCV(
    estimator=mlp,
    param_grid=param_grid,
    cv=5,
    scoring="neg_mean_squared_error",
    n_jobs=max(1,os.cpu_count()//4),
    verbose=1
)

grid_search.fit(model['train_X'], model['train_y'])

print("Best Parameters:", grid_search.best_params_)

Fitting 5 folds for each of 144 candidates, totalling 720 fits
